# Simple Search Model

In class we discuss the solution and identification of this simple model of undirected search. Time is discrete and indexed by $t$ over an infinite horizon. Workers move between employment and unemployment, have linear utility and cannot save. Let us review the parameters of the model:

| Parameter | Description | 
| --------- | ------------|
| $\lambda$ | The probability an unemployed worker receives a job offer | 
| $\delta$  | The probability an employed worker loses their job | 
| $F_{W}$   | The distribution of wage offers | 
| $1-\beta$   | The exponential rate of discounting | 
| $b$       | Per-period utility when unemployed | 

## Model solution

In class we showed that the optimal decision rule of the worker is characterized by a reservation wage. We derived the reservation wage equation:

$$ w^* = b + \beta\lambda\int_{w^*}\frac{1-F_{W}(w)}{1 + \beta(1-\delta)}dw $$

and we characterized the steady state rate of unemployment as:

$$ P[E = 0] = \frac{h}{h+\delta} $$

where $h = \lambda(1-F_{W}(w^*))$ is the rate at which workers exit unemployment. 

Similarly, we should that the steady state fraction of unemployment durations.

Let's write some code to solve the reservation wage equation, starting with code to evalute the equation below:

In [7]:
using Distributions, QuadGK

# this function evaluates the reservation wage equation
res_wage(wres ; b,λ,δ,β,F::Distribution) = wres - b - β * λ * quadgk(x->1-cdf(F,x),wres,Inf)[1]
pars = (;b = 0.5,λ = 0.5,δ = 0.03,β = 0.95,F = LogNormal(1,1))
res_wage(1. ; pars...)

-1.1807330206199615

The most straightforward way to solve for the reservation wage would be to use a root-finding method here. Since we have used `Optim` already, let's just go ahead and use that package:

In [12]:
using Optim
function solve_res_wage(pars)
    (;F) = pars
    w_lb = quantile(F,0.001) #<- get a lower and upper bound for the solution
    w_ub = quantile(F,0.999)
    r = optimize(x->res_wage(x;pars...)^2,w_lb,w_ub)
    return r.minimizer
end
rwage = solve_res_wage(pars)


1.873687807092726